In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breaking-bad-tv-show-all-seasons-episodes-data/breaking_bad.csv


In [2]:
# Install prerequisites
# https://github.com/jmorganca/ollama/issues/1997#issuecomment-1892948729
#pip install llama-index --upgrade
!pip install langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 7.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cu

In [3]:
# Setup the target model globally
#OLLAMA_MODEL='zephyr'
#OLLAMA_MODEL='dolphin2.2-mistral:7b'
#OLLAMA_MODEL='neural-chat:7b'
#OLLAMA_MODEL='tinyllama'
#OLLAMA_MODEL='nous-hermes:7b'
#OLLAMA_MODEL='openhermes2-mistral:7b'
#OLLAMA_MODEL='ifioravanti/neuralbeagle14-7b'
#OLLAMA_MODEL='ifioravanti/alphamonarch'
#OLLAMA_MODEL='gemma:2b'
##OLLAMA_MODEL='gemma:7b'
#OLLAMA_MODEL='adrienbrault/nous-hermes2pro:Q5_K_S'
#OLLAMA_MODEL='dolphin-mistral:v2.8'
OLLAMA_MODEL='llama2'

# Set it at the OS level
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL

llama2


In [9]:
!curl https://ollama.ai/install.sh | sh

# https://github.com/jmorganca/ollama/issues/1997#issuecomment-1892948729
#!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.20#' | sh
!sudo apt install -y neofetch

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13320    0 13320    0     0  42571      0 --:--:-- --:--:-- --:--:-- 42555
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
#######################################################################   99.8%##############          89.7%##################################### 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
E: dpkg was interrupted, you must manually run 'sudo dpkg --configure -a' to correct the problem. 


In [10]:
! sudo dpkg --configure -a

Setting up libjxr-tools (1.2~git20170615.f752187-5) ...
Setting up fonts-urw-base35 (20200910-1) ...
Setting up neofetch (7.1.0-3) ...
Setting up poppler-data (0.4.11-1) ...
Setting up libchafa0:amd64 (1.8.0-1) ...
Setting up libopenexr25:amd64 (2.5.7-1) ...
Setting up libjbig2dec0:amd64 (0.19-3build2) ...
Setting up gsfonts (1:8.11+urwcyr1.0.7~pre44-4.5) ...
Setting up libsixel1:amd64 (1.10.3-3) ...
Setting up libidn12:amd64 (1.38-4ubuntu1) ...
Setting up libimlib2:amd64 (1.7.4-1build1) ...
Setting up netpbm (2:10.0-15.4) ...
Setting up libgc1:amd64 (1:8.0.6-1.1build1) ...
Setting up libfftw3-double3:amd64 (3.3.8-2ubuntu8) ...
Setting up liblqr-1-0:amd64 (0.4.2-2.1) ...
Setting up fonts-droid-fallback (1:6.0.1r16-1.1build1) ...
Setting up libdjvulibre-text (3.5.28-2build2) ...
Setting up libde265-0:amd64 (1.0.8-1ubuntu0.3) ...
Setting up libgs9-common (9.55.0~dfsg1-0ubuntu5.9) ...
Setting up libgs9:amd64 (9.55.0~dfsg1-0ubuntu5.9) ...
Setting up caca-utils (0.99.beta19-2.2ubuntu4) ...


In [11]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
# Let's use fly.io resources
#!OLLAMA_HOST=https://ollama-demo.fly.dev:443
time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 984


In [12]:
!ollama run $OLLAMA_MODEL "Tell me a joke about coding"

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 8934d96d3f08...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕                ▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕           

In [13]:
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings
model = Ollama(model=OLLAMA_MODEL) 

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
#
chain.invoke("Tell me an interesting space fact.")

'\nDid you know that there is a crater on the far side of the Moon named "Shackleton Crater" after the British explorer Ernest Shackleton? This crater is unique because it contains a permanent ice field, which is thought to be one of the largest and most pristine Ice fields in the Solar System. The ice field is located at the bottom of the crater and is estimated to be around 1.5 kilometers thick.\n\nThe Shackleton Crater was named after Ernest Shackleton, who led three British expeditions to the Antarctic between 1907 and 1922. The crater was discovered in 2009 by NASA\'s Lunar Reconnaissance Orbiter (LRO) and has since been studied using a variety of spacecraft and telescopes.\n\nThe discovery of the permanent ice field in the Shackleton Crater is significant because it provides a unique opportunity for scientists to study the formation and evolution of ice in a low-gravity environment. The ice field may also contain clues about the early history of the Moon and the Earth, as well as

In [16]:
file = "/kaggle/input/breaking-bad-tv-show-all-seasons-episodes-data/breaking_bad.csv"

In [18]:
! pip install chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 3.9 MB/s eta 0:00:00a 0:00:01


In [19]:
import chardet
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [21]:
data = pd.read_csv(file,encoding='ISO-8859-1')

In [24]:
data.head()

,Date,Season,Episode,Title,Directed by,Written by,Duration_mins,Summary,Rating_IMDB,U.S. viewers_million
0,20-Jan-08,1,1,Pilot,Vince Gilligan,Vince Gilligan,58,"Diagnosed with terminal lung cancer, chemistry...",9.1,1.41
1,27-Jan-08,1,2,Cat's in the Bag...,Adam Bernstein,Vince Gilligan,48,After their first drug deal goes terribly wron...,8.7,1.49
2,10-Feb-08,1,3,...And the Bag's in the River,Adam Bernstein,Vince Gilligan,48,Walt and Jesse clean up after the bathtub inci...,8.8,1.08
3,17-Feb-08,1,4,Cancer Man,Jim McKay,Vince Gilligan,48,Walt tells the rest of his family about his ca...,8.3,1.09
4,24-Feb-08,1,5,Gray Matter,Tricia Brock,"Vince Gilligan, Patty Lin",48,Walt rejects everyone who tries to help him wi...,8.4,0.97


In [32]:
from langchain.prompts import PromptTemplate 

template = """
Answer the questions based on the context provided. If you are not sure, respond with "I don't know."

Context : {context}
Question: {question}
"""
prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is the question"))


Answer the questions based on the context provided. If you are not sure, respond with "I don't know."

Context : Here is some context
Question: Here is the question



In [33]:
chain = prompt | model | parser

In [34]:
chain.invoke(
    {
        "context": "During my birth, I was named Sanket",
        "question": "What is my current name ?",
    }
)

'Sure! Based on the context provided, your current name is Sanket.'

In [35]:
embeds = OllamaEmbeddings()

In [36]:
! pip install -U docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 6.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.0 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.3 which is incompatible.
rapids-dask-dependency 24.8.0a0 requires dask==2024.7.1, but you have dask 2024.9.0 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.1 which is incompati

In [38]:
from langchain_community.document_loaders import DataFrameLoader
raw_docs = DataFrameLoader(data, page_content_column="Summary").load()

In [41]:
raw_docs[0].page_content

'Diagnosed with terminal lung cancer, chemistry teacher Walter White teams up with former student Jesse Pinkman to cook and sell crystal meth.'

In [42]:
from langchain_community.vectorstores import FAISS

In [44]:
! pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:000:00:010:01m


In [45]:
db = FAISS.from_documents(raw_docs, embeds)

In [ ]:
db.save_local("faiss_index")

In [46]:
retriever = db.as_retriever()

In [47]:
from operator import itemgetter 
chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")}
    | prompt | model | parser
)

chain.invoke({"question": "Roast the resume"})

'Sure! Here\'s my attempt at roasting the resume of the character in Breaking Bad:\n\n"Oh boy, where do I even begin? Let\'s start with the highlights - or should I say, the lowlights. This guy has been involved in some serious shenanigans, from cooking meth to burying bodies. And let me tell you, his work experience is nothing to write home about either. He\'s worked as a high school chemistry teacher, a meth kingpin, and a suburban drug dealer. Talk about a resume gap!\n\nBut wait, it gets better. This guy has also been known to lie, cheat, and steal his way through life. He\'s got a serious problem with authority, and he\'s not afraid to break the law to get what he wants. And let me tell you, his interpersonal skills are lacking at best. He\'s got a tendency to manipulate and deceive those around him, all while maintaining a fake smile on his face.\n\nNow, I know what you\'re thinking - how could anyone with such a shady past get hired by any reputable company? Well, let me tell yo

In [48]:
chain.invoke({"question": "Who is Elon Musk"})

"I don't know who Elon Musk is. The name does not appear in any of the provided documents."

In [102]:
chain.invoke({"question": "Write a new episode for Breaking Bad"})

'Sure! Here is a potential new episode for Breaking Bad:\n\nTitle: "The Cook-Off"\n\nSynopsis: Walt and Jesse are invited to participate in a cook-off competition against other top chefs in the city. Seeing this as an opportunity to prove their skills and gain recognition, they eagerly accept the invitation. However, things take a dark turn when they realize that one of their competitors is not what he seems and will stop at nothing to win. Meanwhile, Skyler becomes suspicious of Walter\'s sudden interest in the cook-off and starts to investigate his secret recipe.\n\nScript:\n\nFADE IN:\n\nEXT. ALBUQUERQUE - DAY\nThe camera pans over the city skyline before zooming in on a bustling street filled with people going about their day. We see Walt and Jesse walking down the street, dressed in their best chef attire.\n\nWALT: (excitedly) This is it, Jesse! Our chance to prove our skills and show everyone what we\'re capable of!\nJESSE: (nervously) Yeah, I guess so. But are we sure we can han

In [53]:
import faiss
# Save FAISS index
faiss.write_index(db.index, "faiss_index.bin")

In [58]:
# storing the data in JSON format
data.to_json('documents.json', orient = 'split', compression = 'infer', index = 'true')

In [60]:
! pip install huggingface_hub

In [67]:
from huggingface_hub import notebook_login

notebook_login()

In [97]:
from huggingface_hub import HfApi

# Authenticate
api = HfApi()

In [96]:
# Upload the folder while maintaining the structure
def upload_folder_with_structure(folder_path, repo_id):
    for root, _, files in os.walk(folder_path):
        for file in files:
            # Create the path for each file
            file_path = os.path.join(root, file)
            # Define the relative path in the repo
            relative_path = os.path.relpath(file_path, folder_path)  # Get relative path from the base folder
            # Upload the file
            api.upload_file(
                path_or_fileobj=file_path,
                path_in_repo=relative_path,
                repo_id=repo_id,
                repo_type="dataset",  # Change to "model" if you're uploading a model
            )
            print(f"Uploaded {relative_path} to {repo_id}")

# Usage
folder_to_upload = "/kaggle/working/"  # Replace with your folder path

upload_folder(folder_to_upload, "sanketsans/Breaking-Bad")

Uploaded faiss_index/index.pkl to sanketsans/Breaking-Bad
Uploaded faiss_index/index.faiss to sanketsans/Breaking-Bad


In [89]:
db.save_local("faiss_index")

In [90]:
FAISS.load_local(
    "faiss_index", embeds, allow_dangerous_deserialization=True
)